In [ ]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
#  Copyright (c) 2023. Mohamed Reda Bouadjenek, Deakin University              +
#           Email:  reda.bouadjenek@deakin.edu.au                              +
#                                                                              +
#  Licensed under the Apache License, Version 2.0 (the "License");             +
#   you may not use this file except in compliance with the License.           +
#    You may obtain a copy of the License at:                                  +
#                                                                              +
#                 http://www.apache.org/licenses/LICENSE-2.0                   +
#                                                                              +
#    Unless required by applicable law or agreed to in writing, software       +
#    distributed under the License is distributed on an "AS IS" BASIS,         +
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  +
#    See the License for the specific language governing permissions and       +
#    limitations under the License.                                            +
#                                                                              +
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Introduction

Welcome to your assignment this week!


To deepen our comprehension of safety in AI, this assignment delves into a practical scenario involving **Image Classification** for traffic signs.





<img src="https://github.com/hashem59/9.1D-AI-Safety/blob/main/images/0_SkE_FOCh8qGP9Zqh.png?raw=1" width="450"/>

In the realm of artificial intelligence, ensuring safety stands as a paramount concern, particularly when the technology is harnessed for critical applications like autonomous driving vehicles. One example is the recognition of traffic signs, where the consequences of inaccurate predictions can reverberate dramatically, potentially resulting in tragic loss of life. In light of these profound implications, this assignment delves into the imperative topic of safety in AI, centering on the vital context of image classification for traffic signals.





Through an examination of safety within the realm of traffic sign classification, our objective is to unveil and mitigate potential risks inherent in such systems. This endeavor is driven by the pursuit of a more profound comprehension of how AI algorithms can inadvertently introduce risks and uncertainties.



Run the following cell to load the packages you will need for this assignment.


In [2]:
import tensorflow as tf
import tensorflow
from tensorflow import keras
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, accuracy_score, classification_report
models = tensorflow.keras.models
layers = tensorflow.keras.layers
optimizers = tensorflow.keras.optimizers
from tensorflow.python.keras.saving import hdf5_format
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
import h5py, itertools, collections
import itertools
from tqdm import trange

##################
# Verifications:
#################
print('GPU is used.' if len(tf.config.list_physical_devices('GPU')) > 0 else 'GPU is NOT used.')
print("Tensorflow version: " + tf.__version__)


ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.12/dist-packages/keras/preprocessing/image/__init__.py)

# Preprocessing


We use a traffic sign dataset that is composed of 48 folders (one for each sign) with 20-500 images in each folder. The total number of images is about 10,000.

For reading these images, we use `DirectoryIterator` in `tf.keras.preprocessing.image` that is an iterator capable of reading images from a directory on disk and is capable to extract labels. We also use `ImageDataGenerator` to split this dataset into training and validation set, this later is used to tune the hyperparameters of our model.


In [ ]:
'''
    Split train and validation.
'''
# We define the size of input images to 64x64 pixels.
image_size = (64, 64)
# We define the batch size
batch_size = 64

# Create an image generator with a fraction of 40% images reserved for validation:
image_generator = ImageDataGenerator(validation_split=0.4)

# Now, we create a training data iterator by creating batchs of images of the same size as
# defined previously, i.e., each image is resized in a 64x64 pixels format.
train_ds =  DirectoryIterator(
    "traffic_sign_dataset/",
    image_generator,
    class_mode='categorical',
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'training',
)

# Similarly, we create a validation data iterator by creating batchs of images of the same size as
# defined previously, i.e., each image is resized in a 64x64 pixels format.
val_ds = DirectoryIterator(
    "traffic_sign_dataset/",
    image_generator,
    class_mode='categorical',
    seed=1337,
    target_size=image_size,
    batch_size=batch_size,
    subset = 'validation',
    shuffle=False
)

# We save the list of classes (labels).
class_names = list(train_ds.class_indices.keys())

# We also save the number of labels.
num_classes = train_ds.num_classes


# Exploring the data

Now, we do data exploration to show you samples of the images and their labels and some statistics to help you in understanding the data.

In [ ]:
###############################################
#### Show distribution of images per class.
###############################################
counter=collections.Counter(train_ds.labels)
v = [ [class_names[item[0]],item[1]]  for item in counter.items()]
df = pd.DataFrame(data=v, columns=['index','value'])
g = sns.catplot(x='index', y= 'value',  data=df, kind='bar',
                legend=False,height=4,aspect=4,saturation=1)
(g.despine(top=False,right=False))
plt.xlabel("Classes")
plt.ylabel("#images")
plt.title("Distribution of images per class")
plt.xticks(rotation='vertical')
plt.show()

#####################################
######### Show sample of images.
#####################################
plt.figure(figsize=(20, 16))
images = []
labels = []
for itr in train_ds.next():
    for i in range(30):
        if len(images) < 30:
            images.append(itr[i].astype("uint8"))
        else:
            labels.append(list(itr[i]).index(1))

for i in range(len(images)):
    ax = plt.subplot(5, 6, i + 1)
    plt.imshow(images[i])
    plt.title(class_names[labels[i]].replace('_',' '))
    plt.axis("off")

plt.show()


# Designing the model


We now design the architecture for the task.

In [ ]:
# Defining your model here:
model = models.Sequential()
model.add(keras.Input(shape=image_size + (3,)))
model.add(layers.experimental.preprocessing.Rescaling(1./255))

## <span style="color:red"><b>TASK 1</b></span>


Define a convolutional neural network (CNN) architecture as follows:

1. CNN Block 1:
    - Add a dropout layer with a dropout rate of 0.25. Dropout is a regularization technique that randomly sets a fraction of input units to 0 during training, which helps prevent overfitting.
    - Add a convolutional layer with 16 filters, each having a 5x5 receptive field. The activation function used is ReLU (Rectified Linear Unit), and "same" padding ensures that the output size remains the same as the input size.
    - Add a max-pooling layer with a 2x2 pool size. Max-pooling reduces the spatial dimensions of the previous layer's output, helping to retain important features while reducing computational complexity.

2. CNN Block 2:
    - Add another dropout layer with a dropout rate of 0.25.
    - Similar to the first convolutional layer, add another convolutional layer with 32 filters, each having a 5x5 receptive field and ReLU activation.
    - Add another max-pooling layer  after the second convolutional layer with a 2x2 pool size.

3. CNN Block 3:
    - Add another dropout layer with a dropout rate of 0.25.

    - Add a third convolutional layer with 32 filters, each having a 5x5 receptive field and ReLU activation.

    - Add another max-pooling layer after the third convolutional layer with a 2x2 pool size.

4. CNN Block 4:    
    - Add another dropout layer with a dropout rate of 0.25.
    - Add a fourth convolutional layer with 32 filters, each having a 5x5 receptive field and ReLU activation.
    - Add another max-pooling layer after the third convolutional layer with a 2x2 pool size.



In [ ]:
## START YOU CODE HERE
pass
## END

In [ ]:
model.add(layers.Flatten())
#Dense part
model.add(keras.layers.Dropout(0.25))
model.add(layers.Dense(num_classes, activation='softmax'))
# Print a summary of the model
model.summary()

# Compiling the model by defininf an optimizer, a loss function,
# and the metrics to be used for monitoring the traning.
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss='CategoricalCrossentropy',
              metrics=['accuracy'])


# Traning

Let's now starting the training process.

In [ ]:
# Start the trining by defining the number of epochs to train, the traing set and the validation set.
history = model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
)


# Monitoring and analysis of the model

The next step consists of monitoring the traning process to investigate possible overfitting.


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(15,4))

ax1 = plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.fill_between(epochs, loss,val_loss,color='g',alpha=.1)

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

ax2 = plt.subplot(1, 2, 2)
plt.plot(epochs, acc, label='Training accuracy')
plt.plot(epochs, val_acc, label='Validation accuracy')
plt.fill_between(epochs, acc,val_acc,color='g',alpha=.1)
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


Run the `classification_report` function below to build a text report showing the main classification metrics for your model:

In [ ]:
val_ds.reset()
val_ds.shuffle = False
val_ds.next()
y_prob = model.predict(val_ds)
y_pred = y_prob.argmax(axis=-1)
y_true = val_ds.labels
print(classification_report(y_true, y_pred, target_names=class_names))

Run the  next cell to create a confusion matrix function `plot_confusion_matrix`.

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    vmax = cm.max()
    if normalize:
        title = 'Confusion matrix (normalized)'
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = [[int(j*100) for j in i ] for i in cm]
        cm =np.array(cm)
        vmax = 100

    plt.figure(figsize=(14,14))

    im = plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0.0, vmax=vmax)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.colorbar(im,fraction=0.046, pad=0.04)


Let's now create the confusion matrix.

In [ ]:
cnf_matrix = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm=cnf_matrix, classes=class_names, title='Confusion Matrix', normalize=True)


## <span style="color:red"><b>TASK 2</b></span>


Examine the confusion matrix and provide an observation-based description of its content and patterns.


***

PLEASE ANSWER HERE!


***


Let's now have a look at miss-classified examples by your model:

In [ ]:
val_ds.reset()
val_ds.shuffle = True
#####################################
######### Show sample of images.
#####################################
plt.figure(figsize=(16, 15))
images = []
labels_pred = []
labels_true = []
while len(images) < 20:
    batch , labels = val_ds.next()
    for i in range(batch.shape[0]):
        y_prob_ = model.predict(np.array([batch[i]]), verbose=0)
        y_pred_ = np.argmax(y_prob_, axis=1)[0]
        y_true_ = list(labels[i]).index(1)
        if y_true_!=y_pred_:
            images.append(batch[i].astype("uint8"))
            labels_pred.append(y_pred_)
            labels_true.append(y_true_)

for i in range(20):
    if labels_pred[i] != labels_true[i]:
        ax = plt.subplot(4, 5, i + 1)
        plt.imshow(images[i])
        title = 'Pred: ' + class_names[labels_pred[i]].replace('_',' ') +'\n' +'True: ' + class_names[labels_true[i]].replace('_',' ')


        plt.title(title,fontsize= 11, pad=5)
        plt.axis("off")


## <span style="color:red"><b>TASK 3</b></span>

Take a moment to relect on the mistakes your model may generate and consider the potential risks these errors pose in a critical application such as an autonomous driving vehicle.

***

PLEASE ANSWER HERE!


***

# Monte carlo Dropout

Monte Carlo Dropout is a technique used in deep learning to estimate uncertainty in predictions made by a neural network. Regular dropout is a technique where during training, random neurons are "dropped out" by setting their outputs to zero. This prevents the network from becoming overly reliant on specific neurons and helps with generalization.

In the context of Monte Carlo Dropout, dropout is applied not only during training but also during inference (when making predictions). Instead of using a single forward pass to make a prediction, Monte Carlo Dropout involves performing multiple forward passes with dropout enabled. By averaging the predictions from these multiple passes, you can obtain a more robust estimate of uncertainty associated with each prediction.

Monte Carlo Dropout provides a way to model and quantify uncertainty in neural network predictions. It's particularly useful in tasks like Bayesian deep learning and uncertainty estimation for tasks such as image classification, where knowing the uncertainty of each prediction can be beneficial.


In [ ]:
val_ds.reset()
val_ds.shuffle = False
# Initialize an empty list to store batches of data
X_val = []

# Iterate through all batches of data
for batch in val_ds:
    # Append the current batch to the list
    X_val.append(batch[0])  # Assuming batch[0] contains the data
    # Break the loop when all batches have been processed
    if val_ds.batch_index == 0:
        break

X_val = np.concatenate(X_val, axis=0)


## <span style="color:red"><b>TASK 4</b></span>

Iterate through the specified number of predictions (200) and utilize dropout when making predictions with the trained model.






In [ ]:
# Initialize an empty list to store predictions
mote_carlo_pred = []
# Loop through the desired number of predictions
for _ in trange(200, ncols=100):
    ## START YOU CODE HERE
    pass
    ## END

## <span style="color:red"><b>TASK 5</b></span>

In our approach, we leverage the concept of entropy as a fundamental tool to estimate uncertainty. By measuring the level of unpredictability within a given set of predictions, entropy provides a quantifiable metric that helps us gauge the uncertainty associated with each prediction. This utilization of entropy enables us to make more informed decisions and predictions in scenarios where uncertainty plays a pivotal role, fostering a deeper understanding of the underlying data dynamics and enhancing the reliability of our outcomes.


In [ ]:
def get_entropy(vector):
    """
    Calculate the entropy of a given vector.

    Parameters:
        vector (list or numpy array): Input data vector.

    Returns:
        float: The computed entropy value.
    """
    ## START YOU CODE HERE
    pass
    ## END
    return entropy


## <span style="color:red"><b>TASK 6</b></span>

Find the most frequent prediction for each set of predictions in mote_carlo_pred and calculate the entropy of each set of predictions, storing both the most frequent predictions and the calculated entropies in separate lists (y_pred_most_freq and entropy, respectively).


In [ ]:
y_pred_most_freq = []
entropy = []
for data in mote_carlo_pred:
    ## START YOU CODE HERE
    pass
    ## END

Let's proceed to contrast the accuracy achieved by our model with that obtained through the utilization of the most frequently predicted value employing Monte Carlo dropout.




In [ ]:
print('Accuracy of our model:', accuracy_score(y_pred, y_true))
print('Accuracy utilizing the most frequently predicted value employing Monte Carlo dropout:',
      accuracy_score(y_pred_most_freq, y_true))


## <span style="color:red"><b>TASK 7</b></span>


What do you conclude from the obtained performance?
***

PLEASE ANSWER HERE!


***

Let's now organize information about predictions, whether they are correct or incorrect, their associated entropy values, and their confidence scores into a Pandas DataFrame for further analysis and visualization.

In [ ]:
confidence_scores = np.max(y_prob, axis=1)
data = {'Prediction': np.array(['Correct' if x == y else 'Incorrect' for x, y in zip(y_pred, y_true)]),
        'Entropy value': entropy,
        'Confidence score': confidence_scores}

df = pd.DataFrame(data)


Let's employ boxplots of the entropy and confidence score distributions to illustrate the distinction between accurate and erroneous predictions.


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Box plot for 'Entropy value'
sns.boxplot(data=df,
            x='Prediction',
            y='Entropy value',
            hue='Prediction',
            order=["Correct", "Incorrect"],
            notch=True,
            showcaps=True,
            flierprops={"marker": "x"},
            ax=axes[0])  # Put it in the first subplot (axes[0])

axes[0].set_xticklabels([])
axes[0].set(xlabel='')
axes[0].set(ylim=(0, None))
axes[0].set_title('Box Plot for Entropy Value')

# Box plot for 'Confidence score'
sns.boxplot(data=df,
            x='Prediction',
            y='Confidence score',
            hue='Prediction',
            order=["Correct", "Incorrect"],
            notch=True,
            showcaps=True,
            flierprops={"marker": "x"},
            ax=axes[1])  # Put it in the second subplot (axes[1])

axes[1].set_xticklabels([])
axes[1].set(xlabel='')
axes[1].set(ylim=(0, None))
axes[1].set_title('Box Plot for Confidence Score')

# Adjust the layout
plt.tight_layout()

# Show the subplots
plt.show()


## <span style="color:red"><b>TASK 8</b></span>

What do you conclude from the obtained results?
***

PLEASE ANSWER HERE!


***


Let's proceed to visualize the achieved accuracy against the proportion of predicted examples from the validation set, considering different entropy cutoff values.



In [ ]:
accuracy = []
size = []
for v in [x * 0.01 for x in range(1, 410)]:
    df2 = df[(df['Entropy value']<v)]
    count = len(df2[df2['Prediction'] == 'Correct'])
    accuracy.append(count/len(df2))
    size.append(len(df2)/len(df))
# Create a line plot
plt.plot(size, accuracy, marker='o', linestyle='-')

# Set labels for x and y axes
plt.ylabel('Accuracy')
plt.xlabel('Size')

# Set a title for the plot
plt.title('Accuracy vs. Size')

# Show the plot
plt.grid(True)  # Add grid lines
plt.show()

## <span style="color:red"><b>TASK 9</b></span>

What do you conclude from the obtained results?

***

PLEASE ANSWER HERE!


***


## <span style="color:red"><b>TASK 10</b></span>

Could you propose an alternative technique that harnesses uncertainty to enhance the model's overall performance?


***

PLEASE ANSWER HERE!


***


# Acknowledgment

Please feel free contact me if you identify errors, bugs, or issue with this assignment.

**Author:** [Mohamed Reda Bouadjenek](https://rbouadjenek.github.io/), Lecturer of Applied Artificial Intelligence,

**Institution:** Deakin University, School of Information Technology, Faculty of Sci Eng & Built Env

**Adress:** Locked Bag 20000, Geelong, VIC 3220

**Phone:** +61 3 522 78380

**Email:** reda.bouadjenek@deakin.edu.au

**www.deakin.edu.au**
